### Importing the libraries

In [1]:
import json 
import requests 
import pandas as pd

In [2]:
api_token = 'wR6uVuJhdEqHqCn7f6ZOKaHAy2mK2AfYCeAwVqCb'
workspace_id = 'c2f4db82-a46e-4128-862f-f0531ce8de03'

auth_url = 'https://eu-svc.leanix.net/services/mtm/v1/oauth2/token' 
request_url = 'https://eu-svc.leanix.net/services/mtm/v1/workspaces/%s/users?page=0'%(workspace_id) 

# Get the bearer token - see https://dev.leanix.net/v4.0/docs/authentication
response = requests.post(auth_url, auth=('apitoken', api_token),
                         data={'grant_type': 'client_credentials'})
response.raise_for_status() 
access_token = response.json()['access_token']
auth_header = 'Bearer ' + access_token
header = {'Authorization': auth_header}

### Function to call the given query 

In [3]:
def call():
  response = requests.get(url=request_url, headers=header)
  response.raise_for_status()
  return response.json()

In [4]:
email_list, first_name_list, last_name_list, last_login_list = ([] for i in range(4))

In [5]:
funct = call()

### Loop to find out the users data

In [6]:
for user in funct['data']:
    first_name_list.append(user['firstName'])
    last_name_list.append(user['lastName'])
    email_list.append(user['email'])

In [7]:
df = pd.DataFrame()

### Defining columns for the dataframe using the lists:

In [8]:
df['First_Name'] = first_name_list
df['Last_Name'] = last_name_list
df['Email'] = email_list

In [9]:
df = df.set_index('First_Name')

In [10]:
first = []
current_login = []

In [11]:
for user in funct['data']:
    print(len(user.keys()))

15
15
15
15
15
15
15
15
19
17
15
15
17


Some of the users data doesn't have a current login time. This can be seen in the length of the keys. If the user length is 15 or less, then that user does not have the current login data. The code below shows to extract the current login data from the user.

In [12]:
for user in funct['data']:
    if len(user.keys())>15:
        first.append(user['firstName'])
        current_login.append(user['currentLogin'])

In [13]:
df1 = pd.DataFrame()
df1['First_Name'] = first
df1['Current_Login'] = current_login

In [14]:
df1 =df1.set_index('First_Name')

### Joining the data frame with the first name as the index 

In [15]:
df =df.join(df1)

In [16]:
df = df.fillna('Not Available')

### Saving the data frame into csv

In [18]:
df.to_csv('output.csv')

In [19]:
df =pd.read_csv('output.csv')

### The output:

In [22]:
df.head(13)

,First_Name,Last_Name,Email,Current_Login
0,Nicola,Martin,nicola.PRIDemo@meshlab.de,Not Available
1,Richard,Best,srv.PRIDemo@meshlab.de,Not Available
2,Carol,Brown,pgm.PRIDemo@meshlab.de,Not Available
3,Peter,Schmidt,cio.PRIDemo@meshlab.de,Not Available
4,Paula,Watson,mgr.PRIDemo@meshlab.de,Not Available
5,Christina,Weber,christina.PRIDemo@meshlab.de,Not Available
6,Max,Sander,max.PRIDemo@meshlab.de,Not Available
7,Benjamin,Parker,benjamin.PRIDemo@meshlab.de,Not Available
8,LeanIX,Support,support@leanix.net,2022-04-01T11:44:00.248Z
9,Francois,Kruger,francois.kruger@leanix.net,2022-04-01T09:22:47.671Z
